In [34]:
#%% 
import sys 
sys.path.append("C:/Users/win10/Workspace/kaggle.gongdeok/iceberg_week1/paul_temp")

import numpy as np

import pandas as pd

from scipy import signal
from random import sample
from iceberg_data import load_data, to_numpy, load_test
import models

import tensorflow as tf 
import tensorflow.keras.layers as L
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
plt.style.use('ggplot')

tf.executing_eagerly()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus: tf.config.experimental.set_memory_growth(gpus[0], True)

In [2]:
data = load_data()

mask = data['inc_angle'].isnull()
train = data[~mask]
test  = data[mask]
len(train), len(test)

(1471, 133)

In [4]:
band_1 = train['band_1'].apply(lambda x: np.reshape(x, newshape=(75,75)))
band_2 = train['band_2'].apply(lambda x: np.reshape(x, newshape=(75,75)))
X = np.array([
        np.transpose(np.array([b1,b2]),[1,2,0]) 
        for b1, b2 in zip(band_1, band_2)], dtype=np.float32)
Y = np.array(train['inc_angle'], dtype=np.float32)

In [5]:
"Build Dataset"
batch_size = 1
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.25)
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(batch_size).shuffle(10000)
val_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(len(y_train)).shuffle(10000)

In [6]:
model = tf.keras.models.Sequential([
    L.Conv2D(4*16, kernel_size=(3, 3),activation='relu'),
    L.MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
    L.Dropout(0.2),
    L.Flatten(),
    L.Dense(32),
    L.Dense(1)])

model.build((1,75,75,2))
model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=10e-4),
    loss=tf.keras.losses.mae,
#     metrics=[log_loss],
)


In [7]:
check_pointer = ModelCheckpoint(
    filepath='./models/inc_angle.h5',
    monitor='val_loss',
    verbose=0,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',
    save_freq='epoch',)

history = model.fit(
    dataset, 
    validation_data=val_dataset, 
    epochs=250,
    callbacks=[check_pointer])

Epoch 1/250
1103/1103 [==============================] - 7s 6ms/step - loss: 16.8762 - val_loss: 0.0000e+00
Epoch 2/250
1103/1103 [==============================] - 3s 3ms/step - loss: 6.2363 - val_loss: 7.0191
Epoch 3/250
1103/1103 [==============================] - 3s 3ms/step - loss: 4.5586 - val_loss: 3.0514
Epoch 4/250
1103/1103 [==============================] - 3s 3ms/step - loss: 4.5507 - val_loss: 2.4891
Epoch 5/250
1103/1103 [==============================] - 3s 3ms/step - loss: 3.8700 - val_loss: 3.1706
Epoch 6/250
1103/1103 [==============================] - 3s 3ms/step - loss: 3.8545 - val_loss: 4.5268
Epoch 7/250
1103/1103 [==============================] - 3s 3ms/step - loss: 3.4991 - val_loss: 3.3210
Epoch 8/250
1103/1103 [==============================] - 3s 3ms/step - loss: 3.1663 - val_loss: 2.5073
Epoch 9/250
1103/1103 [==============================] - 3s 3ms/step - loss: 3.4330 - val_loss: 5.0094
Epoch 10/250
1103/1103 [==============================] - 3s 3ms/ste

KeyboardInterrupt: 

In [13]:
val_dataset = tf.data.Dataset.from_tensor_slices((X_valid, y_valid)).batch(len(y_train))
model.load_weights('./models/inc_angle.h5')
model.evaluate(val_dataset)

1/1 [==============================] - 0s 30ms/step - loss: 2.3976


2.397649049758911

In [15]:
band_1 = test['band_1'].apply(lambda x: np.reshape(x, newshape=(75,75)))
band_2 = test['band_2'].apply(lambda x: np.reshape(x, newshape=(75,75)))
X_t = np.array([
        np.transpose(np.array([b1,b2]),[1,2,0]) 
        for b1, b2 in zip(band_1, band_2)], dtype=np.float32)

In [21]:
Y_p = model.predict(X_t).squeeze()

In [28]:
# Fill inc_angle Null data

In [40]:
data = load_data()

mask = data['inc_angle'].isnull()
data.loc[mask, 'inc_angle']= Y_p
data.isnull().any()

id            False
band_1        False
band_2        False
inc_angle     False
is_iceberg    False
dtype: bool

In [41]:
data.to_json("../../../kaggle.dataset/statoil-iceberg-classifier-challenge/train_nona.json")